# Analyse de tableaux de données avec pandas

## 1 Introduction
Vous aurez besoin d’au moins 5 Go de RAM disponible pour la suite de ce TD. Si vous comptiez utiliser votre ordinateur personnel mais que ce dernier ne dispose pas de 5 Go de RAM disponible, utilisez les ordinateurs de l'école.

Le notebook contient à la fois des parties plutôt "tutoriel" : ces parties vous montrent les capacités de pandas, et vous demandent d'executer des cellules de code pré-remplies. Si l'énoncé vous fournit le résultat attendu, vérfiez simplement que le résultat que vous obtenez est correct.
Le notebook contient également des questions ainsi qu'un exercice final dans lesquels c'est à vous de trouver et ajouter les lignes de code répondant au problème !


### 1.1 Objectif
L’objectif de cette séance est de se familiariser avec la manipulation et le traitement de (potentiellement gros) tableaux de données en python, pour en ressortir une information précise. Par tableau on entend “une liste d'éléments (les lignes du tableau) possédant tous les mêmes attributs (les colonnes du tableau)”, à l’instar de ce que l’on peut trouver dans un classeur Excel ou une table de base de données SQL.
Pour ce TD, nous utiliserons le package python pandas en chargeant des tableaux depuis des fichiers csv (Comma Separated Values: fichier texte contenant des valeurs séparées par des virgules). A noter que pandas peut également charger des tableaux depuis des bases de données SQL ou fichiers Excel (non abordé ici).

### 1.2 Présentation et récupération des données
Les données que nous allons manipuler dans ce TD est un ensemble de fichiers csv (ou plus précisément “tsv” car le séparateur est le caractère “tab” et non pas une virgule) mis à disposition gratuitement par IMDB.com [sur cette page](https://datasets.imdbws.com/). La description de chaque fichier est disponible [ici](https://developer.imdb.com/non-commercial-datasets/).

Créez un dossier IMDB. téléchargez (ou copiez) les fichiers tsv compréssés (avec l'extension gz) title.ratings, title.basics, title.principals et name.basics dans le dossier IMDB Décompressez-les dans ce dossier avec l'executable gzip : 

```
ls *.gz |xargs -n1 gzip -dv
```

Une fois cela fait, en executant la cellule ci-dessous, vous devriez obtenir le résultat suivant (au nom d'utilisateur près):
```
drwxr-xr-x 2 clement clement       4096 Oct  1 00:37 ./
drwxr-xr-x 8 clement clement       4096 Sep 30 22:14 ../
-rw-r--r-- 1 clement clement  906244603 Sep 30 22:14 name.basics.tsv
-rw-r--r-- 1 clement clement 1033130568 Sep 30 22:15 title.basics.tsv
-rw-r--r-- 1 clement clement 4235431938 Sep 30 22:15 title.principals.tsv
-rw-r--r-- 1 clement clement   28201022 Sep 30 22:15 title.ratings.tsv

```

In [ ]:
%%bash
# Executez cette cellule sans la modifier et vérifiez que le résultat est correct
ls -l IMDB/

## 2 Chargement avec pandas
### 2.1 Chargement d’un csv simple (10 min)
Commençons par charger un premier csv dans un DataFrame pandas grâce à sa fonction read_csv.

In [ ]:
import pandas
ratings = pandas.read_csv("IMDB/title.ratings.tsv")

Affichez un aperçu de ce DataFrame en éxecutant la cellule suivante :

In [ ]:
ratings

Vous constaterez que pandas n’a pas interprété les tabs présents dans le fichier comme des
séparateurs de colonne.

**Q1)** En vous aidant de la documentation de la fonction read_csv, ajoutez l’option adéquate afin de
charger correctement le csv et ainsi obtenir cet aperçu :
```
	tconst	averageRating	numVotes
0	tt0000001	5.7	2089
1	tt0000002	5.6	283
2	tt0000003	6.5	2096
3	tt0000004	5.4	183
4	tt0000005	6.2	2832
...	...	...	...
1482925	tt9916730	7.0	12
1482926	tt9916766	7.1	24
1482927	tt9916778	7.2	37
1482928	tt9916840	7.2	10
1482929	tt9916880	8.6	8
1482930 rows × 3 columns
```

In [ ]:
# Solution Q1)
# insert your code here


Vous pouvez constater que pandas a automatiquement interprété le type de chaque colonne
(float, int, …) en affichant l’attribut dtypes du DataFrame (par défaut, pandas stock les chaines de
caracteres dans un champs de type object) :

In [ ]:
ratings.dtypes

### 2.2 Chargement d’un csv plus complexe
Chargez maintenant le csv “title.basics.tsv” dans un DataFrame nommé basics, de la même manière que pour “title.ratings.tsv”.

NB : Pour les très gros fichiers, il peut être judicieux d'indiquer à pandas de ne pas etre trop gourmand en RAM grâce à l'option low_memory=True

In [ ]:
basics = pandas.read_csv("IMDB/title.basics.tsv", sep="\t", low_memory=True)
basics

Si vous affichez les types déduits par pandas pour chaque colonne, vous constaterez qu'il n’a pas réussi à inférer le bon type des colonne isAdult (colonne 4) qui devrait etre reconnue comme booléen (ou int),
startYear, endYear et runtimeMinutes (normalement des int).

In [ ]:
basics.dtypes

La première raison pour laquelle pandas interprete mal le contenu du fichier vient de la manière que IMDB utilise pour indiquer qu’un champ n’est pas
renseigné. En effet, comme ils le spécifient sur [la page de présentation des données](https://developer.imdb.com/non-commercial-datasets) il utilisent la
chaine de caractère “\N”, qui ne fait pas partie des symboles par défaut que pandas interprète automatiquement en tant que champs vide (la liste par défaut est renseignée dans la documentation de l’option na_values de la fonction read_csv).

**Q2)** En utilisant l’option na_values de read_csv, rechargez le csv basics. **Attention, comme le dataframe basics que vous avez chargé à la question précédente occupe beaucoup de mémoire vive, mieux vaut le supprimer avant d’essayer de le recharger ! Pour cela, utilisez l’instruction del.**

In [ ]:
del basics

In [ ]:
# Solution Q2)
# insert your code here


Remarquez l’avertissement :

```DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.```

pandas indique maintenant un problème à la colonne 7.

Si vous affichez le type inféré par pandas pour chaque colonne, vous constaterez en effet qu’il a réussi sauf pour la colonne 7 (runtimeMinutes) :

In [ ]:
print(basics.dtypes)

Pour comprendre quelles entrées dans la base de données posent problème à pandas, on peut tenter de forcer pandas à convertir la colonne runtimeMinutes (qui contient des "Mixed types") vers le type "float", en esperant qu'un message d'erreur nous donne plus de détails sur le probleme.

In [ ]:
basics.runtimeMinutes.astype(float)

Vous devez avoir obtenu une erreur avec le message suivant : 
```
ValueError: could not convert string to float: 'Reality-TV'
```
Interessons-nous donc davantage aux entrées pour lesquelles la colonne runtimeMinutes vaut "Reality-TV"

In [ ]:
basics[basics.runtimeMinutes == "Reality-TV"]

On constate que pandas parse ces lignes de maniere erronnée dès la colonne originalTitle ! Et en regardant bien la colonne primaryTitle, on constate que la valeur qui aurait du etre dans originalTitle semble avoir été concatenée à tort dans la colonne primaryTitle.
Pour essayer de comprendre pourquoi, on peut afficher la ligne brute du fichier tsv associée à une de ces entrées de la BDD. Pour cela, sur un systeme Unix (Linux ou Mac), on peut utiliser le programme "grep" :

In [ ]:
%%bash
grep -e tt10233364 IMDB/title.basics.tsv 

Un oeil averti verra alors immediatement le problème : les colonnes primaryTitle et originalTitle commencent toutes les eux par un guillement qui n'est pas refermé ! 
Hors, pandas ignore tous les caracteres speciaux (y compris les caractères servant de separateur de colonne !) compris entre deux guillements. Pour lui, le tab présent à la fin de la 1ere occurence de "Deep Dish" fait donc partie integrante de la colonne primaryTitle et n'est pas interprété comme un séparateur de colonne !

**Q3)** Pour remédier à cela, recharger le csv en utilisant l’option quoting=csv.QUOTE_NONE de la fonction read_csv, qui permet de ne donner aucun sens particulier au caractère guillemet. Comme pour la précédente question, pensez à bien détruire le dataframe basics avant de le recharger pour économiser de l'espace en RAM.

In [ ]:
# Solution Q3)
# insert your code here


Vous pouvez maintenant constater que pandas a correctement interprété les types de chaque colonne : 

In [ ]:
basics.dtypes

## 3 Accès aux éléments d’un DataFrame
### 3.1 df.loc[] , df.iloc[] , df[]
Les DataFrame possèdent deux dimensions. L’attribut “loc” d’un DataFrame permet d’adresser la
première (les lignes) via un nom d’index et la seconde (les colonnes) via un nom de colonne.
Vous pouvez afficher la liste des index et des colonnes via les attributs “index” et “colonne” d’un
DataFrame :

In [ ]:
ratings.index

In [ ]:
ratings.columns

In [ ]:
basics.loc[42, "startYear"]

Il est possible d'accéder à plusieurs colonnes pour un index donné, ou plusieurs index pour une
colonne donné, auquel cas pandas retournera une Series :

In [ ]:
s = basics.loc[42, ["originalTitle", "startYear"]]
print(s)
print(f"{type(s)=}")

In [ ]:
s = basics.loc[[42, 2023], "originalTitle"]
print(s)
print(f"{type(s)=}")

Il est possible d’extraire plusieurs index et colonnes à la fois, auquel cas pandas retournera un
DataFrame :

In [ ]:
d = basics.loc[[42, 2023], ["originalTitle", "startYear"]]
print(d)
print(f"{type(d)=}")

Il est également possible de faire les mêmes accès à des éléments/lignes/colonnes/sous-DataFrame via les numéros de lignes et colonnes grâce à l’attribut iloc :

In [ ]:
basics.iloc[[42, 2023], [2,7]]

Vous pouvez utiliser le symbole “:” sur une des dimensions de l’attribut loc ou iloc afin de sélectionner tous les champs d’une dimension et ainsi récupérer une ligne ou colonne entière.

In [ ]:
ratings.loc[:, "numVotes"]

In [ ]:
basics.loc[5, :]

Dans le cas des colonnes, il existe une possibilité supplémentaire : l'opérateur [] ou “.” directement appliqué au DataFrame :

In [ ]:
basics.startYear

In [ ]:
basics["titleType"]

## 4 Opérations sur les colonnes
### 4.1 Opérations arithmétiques entre colonnes

In [ ]:
ratings.numVotes * ratings.averageRating

### 4.2 Statistiques

In [ ]:
N = ratings.numVotes.sum()
moy = ratings.averageRating.mean()
std = ratings.averageRating.std()

La méthode describe permet d’avoir rapidement des informations sur la distributions des valeurs dans une colonne de valeurs numériques :

In [ ]:
ratings.describe()

### 4.3 Application de fonctions numpy

In [ ]:
import numpy as np
np.mod(basics.startYear, 10)

### 4.4 Application de fonctions personnalisées

In [ ]:
basics.startYear.apply(lambda x: x/2 if x%2==0 else 2*x+1)

## 5 Ajout d’une colonne dans un DataFrame

In [ ]:
basics["runtimeHours"] = basics["runtimeMinutes"] / 60
basics["isOld"] = basics["startYear"] < 1992
basics[["originalTitle", "runtimeMinutes", "runtimeHours", "startYear", "isOld"]]

## 6 Filtrage / Indexage logique

Directement avec l'opérateur DataFrame[] : 

In [ ]:
ratings[ratings.numVotes > 1e6]

Ou bien avec l'opérateur DataFrame.loc[] : 

In [ ]:
basics.loc[basics.startYear > 2020, "runtimeMinutes"]

**Q4)** Extraire dans un DataFrame “movies” les éléments de basics dont l’attribut “titleType” est “movie”.

In [ ]:
# Solution Q4)
# insert your code here


## 7 Fusion de DataFrame
Il est possible de croiser les informations de deux DataFrame différents. Pour cela, pandas possède
entre autres la fonction merge.

**Q5)** Après avoir étudié la documentation de la fonction merge, créer le DataFrame “rated_movies”
qui regroupe les informations des DataFrames “movies” et “ratings” uniquement pour les films
possédant une note sur IMDB.

Les cinq premiers éléments de ce DataFrame doivent être ceux-ci :
``` 
   averageRating                   primaryTitle
0            5.4                     Miss Jerry
1            5.2  The Corbett-Fitzsimmons Fight
2            4.4                       Bohemios
3            6.0    The Story of the Kelly Gang
4            5.7               The Prodigal Son
5            4.3             Robbery Under Arms
```

In [ ]:
# Solution Q5)
# insert your code here


## 8 Regroupement de lignes
Il est possible de regrouper les lignes d’un DataFrame qui possède la même valeur pour une
colonne donnée, et d’ensuite extraire des statistiques sur ces groupes.

In [ ]:
movies.groupby('startYear').tconst.count()

In [ ]:
movies.groupby('startYear').runtimeMinutes.mean()

## 9 Tracé de graphes
### 9.1 Histogramme
La méthode hist des DataFrame permet de tracer l’histogramme de toutes les colonnes à valeurs numériques du DataFrame.

In [ ]:
ratings.hist(bins=100)

On peut également l’appeler sur une colonne en particulier :

In [ ]:
np.log10(ratings.numVotes).hist(bins=100)

### 9.2 Courbes
On peut simplement tracer les valeurs d’une colonne en fonction d’une autre colonne grâce à la méthode “plot” de DataFrame. L’option “kind” permet de choisir le type de graphe.

In [ ]:
rated_movies.plot("startYear", "numVotes", kind="scatter")

### 9.3 Pie Charts

In [ ]:
basics.groupby("titleType").titleType.count().plot.pie()

## 10 Filtrages de csv volumineux

Comme vous pouvez le constater, certains fichiers sont très gros : jusqu’à 2.7 Go pour la table “title.principals.tsv” listant les acteurs principaux de l’ensemble des titres de IMDB. Cela peut rendre long, voire impossible en fonction de la quantité de mémoire vive que possède votre ordinateur, le chargement de ces tables en RAM.
Pour la suite du TD, afin de ne pas avoir à manipuler l'intégralité de la table title.principal.tsv, nous allons la filtrer pour ne garder que les lignes associées à un élément de type “movie”. Nous écrirons le résultat de ce filtrage dans un fichier tsv plus petit “movie.principals.tsv” afin de pouvoir le recharger facilement ultérieurement.

### 10.1 Filtrage par morceaux
Avec l’option “chunksize” de la fonction “read_csv”, il est possible de charger un gros fichier csv petit bout par petit bout. Ainsi on peut filtrer chaque petit bout en stockant le résultat de ce filtrage partiel dans une liste temporaire avant de finalement concatener tous ces morceaux filtrés en un unique DataFrame :

In [ ]:
from tqdm import tqdm
subchunks = []
movies_ids = movies.tconst.values
with pandas.read_csv("IMDB/title.principals.tsv", sep="\t", na_values=["\\N"], quoting = csv.QUOTE_NONE, chunksize=100000) as reader:
    for k, chunk in tqdm(enumerate(reader)):
        subchunk = chunk[chunk.tconst.isin(movies_ids)]
        subchunks.append(subchunk)
movie_principals_list = pandas.concat(subchunks)

### 10.2 Ecriture de la table dans un fichier
La méthode “to_csv” des DataFrame permet de simplement écrire un fichier csv :

In [ ]:
movie_principals_list.to_csv("IMDB/movie.principals.tsv", index=False, sep="\t", na_rep="\\N", quoting=csv.QUOTE_NONE)

**Q6)** A partir des morceaux de code précédents, créer le DataFrame “movies_actor_list” et le fichier “movie.actors.tsv”, qui ne contiennent que les informations de “title.principals.tsv” concernant **les rôles d’acteurs/actrices dans des films de cinéma**.

In [ ]:
# Solution Q6) 
# insert your code here


**Q7)** A partir du fichier “name.basics.tsv”, créer un DataFrame “movie_actors_basics”, que vous écrirez dans un fichier “movie_actors.basics.tsv”, ne contenant que les informations personnelles relatives aux acteurs/actrices ayant joué dans un film de cinéma (c'est à dire les acteurs/actrices apparaissant au moins une fois dans le dataframe movies_actor_list)

In [ ]:
# Solution Q7)
# insert your code here


# 11 Exercices

1) Combien de films ont obtenu la note de 10/10 ?
2) Afficher l’histogramme des notes des films sortis ces 10 dernières années.
3) Donner les noms, les nombres de votes et les notes moyennes des 5 films les mieux notés et ayant plus de 50000 notes.
4) Donner les noms, notes moyennes, nombres de votes et dates de sortie des 5 comédies avec plus de 50000 notes les mieux notées de ces 20 dernières années.
5) Donner la moyenne et écart-type des notes par genre de film (indice : vous pourrez regarder la méthode Series.str.split et DataFrame.explode).
6) Tracer l'évolution de la note moyenne des films d’action en fonction de l’année de sortie.
7) Donner la liste des 5 acteurs vivants ayant joué dans le plus de films.
8) Donner la liste des 5 acteurs ayant joué dans au moins 5 films, avec la filmographie la mieux notée.
9) Tracer le graphe “Nombre de films joués” versus “Note moyenne de la filmographie”.

In [ ]:
# Solution exo 1)
# insert your code here


In [ ]:
# Solution exo 2)
# insert your code here


In [ ]:
# Solution exo 3)
# insert your code here


In [ ]:
# Solution exo 4)
# insert your code here


In [ ]:
# Solution exo 5)
# insert your code here


In [ ]:
# Solution exo 6)
# insert your code here


In [ ]:
# Solution exo 7)
# insert your code here


In [ ]:
# Solution exo 8)
# insert your code here


In [ ]:
# Solution exo 9)
# insert your code here


# 12 Exercice final

Créez un utilitaire en ligne de commande qui, lorsqu'executé depuis un terminal, affiche un message d'accueil et demande à l'utilisateur de choisir parmi une des fonctionnalités suivantes : 
    a) affichage d'un profil d'acteur/actrice
    b) affichage du top 5 des films par genre et par année

Dans le cas ou l'utilisateur a fait le choix a), l'utilitaire doit demander à l'utilisateur de renseigner un nom d'acteur. 
* Si le nom renseigné existe bien dans la base, l'utilitaire doit afficher :
  * les infos sur l'acteur/actrice (date de naissance, age, ...)
  * la note moyenne de sa filmographie
  * un histogramme montrant le nombre de films dans lequel la personne a joué en fonction des années
* Dans le cas ou le nom ne correspond à aucun acteur/Actrice, l'utilitaire propose les 5 noms les plus proches.
* Apres avoir afficher tout cela, l'utilitaire revient à l'accueil.


Dans le cas ou l'utilisateur a fait le choix b), l'utilitaire doit afficher la liste de tous les genres disponibles dans la base, et demander à l'utilisateur d'en choisir un parmi eux. Puis l'utilitaire demande à l'utilisateur de renseigner une année. 
* Dans le cas ou il existe un ou plusieurs films du genre selectionné pour l'année renseignée, l'utilitaire affiche les (au plus) 5 films les mieux notés de ce genre pour cette année.
* Dans le cas ou aucun film de ce genre n'a été réalisé cette année là, l'utilitaire demande à l'utilisateur de renseigner une autre année.
* Apres avoir afficher, l'utilitaire revient à l'accueil.